In [0]:
import os
import time
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import quote

#取得博客來的ISBN

def get_books_ISBN():
    
    list_isbn = []
    url='https://www.books.com.tw/web/sys_tdrntb/books/?loc=menu_th_1_001'
    re = requests.get(url)
    soup = BeautifulSoup(re.text,'html.parser')
    
    url2 = []
    
    count=0-
    while count<10:
        new_url = soup.findAll('li', {'class':'item'})[count].a['href']
        url2.append(new_url)
        count+=1
    
    for i in url2:
        re2 = requests.get(i)
        soup2 = BeautifulSoup(re2.text, 'html.parser')
        try:
            isbn = soup2.find(attrs = {'itemprop':'productID'})['content'][5::]
            list_isbn.append(isbn)
        except TypeError:
            list_isbn.append('N/A')
        re2.close()
        time.sleep(10)
    
    return list_isbn

books_ISBN=get_books_ISBN()
print (books_ISBN)

SyntaxError: ignored

In [0]:
#取得誠品排行榜的ISBN
def get_eslite_ISBN():
    url_eslite = 'http://www.eslite.com/saleboard.aspx'
    res_eslite = requests.get(url_eslite)
    soup_eslite = BeautifulSoup(res_eslite.text, 'html.parser')

    eslitelist_booklink=[]
    eslite_ISBN=[]
    results=soup_eslite.select("div.SB_top10")

    for item in results:
        item_titles = item.select("a")
  
        for title in item_titles:
            item_booklink = title.get("href")
            eslitelist_booklink.append(item_booklink)       

    count=0               
    for urllink in eslitelist_booklink:
        if count<10:
            res_eslite2 = requests.get(urllink)
            soup_eslite2 = BeautifulSoup(res_eslite2.text, 'html.parser')
  
            ISBN_eslite = soup_eslite2.findAll('div',{'class': 'C_box'})
            listk= ISBN_eslite[-1].p.text
            listk2= listk.split()
  
            eslite_ISBN.append(listk2[4][1:])
            count+=1
        else:
            break
    return eslite_ISBN

#出示誠品排行榜前十ISBN為一串列
eslite_ISBN=get_eslite_ISBN()
print(eslite_ISBN)

['9789862139318', '9789861365145', '9789576580833', '9789864796038', '9789579164337', '9789867778444', '9789869593045', '9789867778178', '9789861336558', '9789570852387']


In [0]:
#取得金石堂的ISBN

def get_kingstone_ISBN():
    headers = {'user-agent':'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.94 Safari/537.36'}

    url_kingstone = 'https://www.kingstone.com.tw/book/book_board.asp?RankType=week&selltype=sell01'
    res_kingstone = requests.get(url_kingstone,headers=headers)
    soup_kingstone = BeautifulSoup(res_kingstone.text, 'html.parser')

    kingstonelist_booklinkyet=[]
    kingstonelist_ISBN=[]

    results_kingstone = soup_kingstone.select("li.rise")

    for item in results_kingstone:
        item_titles_k = item.select("a")
  
        for title in item_titles_k:
            item_booklink = title.get("href")
            kingstonelist_booklinkyet.append("https://www.kingstone.com.tw" + item_booklink) 
        
      
    kingstonelist_booklink= kingstonelist_booklinkyet[0:30:3]

    for kurllink in kingstonelist_booklink:
        res_kingstone2 = requests.get(kurllink,headers=headers)
        time.sleep(1)
        soup_kingstone2 = BeautifulSoup(res_kingstone2.text, 'html.parser')
    
        try:
            description = soup_kingstone2.find(attrs={"name":"description"})['content']
            keywords = soup_kingstone2.find(attrs={"name":"keywords"})['content']
        
        except TypeError:
            description ="書名：18禁不顯示，語言：中文繁體，ISBN:xxxxxxxxxxxxx，出版社:不顯示，作者:不顯示，出版日期:不顯示，類別：不顯示"      
            index = description.find('ISBN:')
            kingstonelist_ISBN.append(description[index+5 : index+18])
        
            continue
      
        index = description.find('ISBN:')
        kingstonelist_ISBN.append(description[index+5 : index+18]) 
   
    return kingstonelist_ISBN

#出示金石堂排行榜前十ISBN為一串列，若為18禁則出示xxxxxxxxxxxxx
kingstone_ISBN=get_kingstone_ISBN()
print(kingstone_ISBN)

['9789862717752', '9789861365145', '9789864061419', 'xxxxxxxxxxxxx', '9789863616108', '9789867778444', '9789575646196', 'xxxxxxxxxxxxx', '9789576580833', '9789865695859']


In [0]:
#輸入ISBN得到書名簡介

def checkprofile(key):
    alist=[]

    url = 'http://www.eslite.com/Search_BW.aspx?query='+key
    re = requests.get(url)
    re.encoding = 'utf-8'
    soup = BeautifulSoup(re.text, 'html.parser')

    first_result = soup.find('td', {'class': 'name'})
    try: 
        url2 = 'http://www.eslite.com/'+first_result.h3.a['href'] 
    
    except AttributeError:
        url = 'https://search.books.com.tw/search/query/key/'+key+'/cat/BKA'
        re = requests.get(url)
        re.encoding = 'utf-8'
        soup = BeautifulSoup(re.text, 'html.parser')

        first_result = soup.find('div', {'class': 'input_buy'})
        try: 
            url2= 'https://www.books.com.tw/products/'+first_result.input['value']
    
        except AttributeError:
            return['NA','NA','NA','NA','NA','NA']
        
        author= soup.find('a', {'rel': 'go_author'})

        re2 = requests.get(url2)
        re2.encoding = 'utf-8'
        soup2 = BeautifulSoup(re2.text, 'html.parser')

        profile = soup2.find('div', {'class': 'container_24 main_wrap clearfix'}).find('div',{'class':'grid_10'})
        name=profile.h1.text
    
        basic_profile=profile.find('div', {'class': 'type02_p003 clearfix'})
        price_profile=profile.find('div', {'class': 'cnt_prod002 clearfix'})
        basic=basic_profile.find_all('li')
        price=price_profile.find('li')

        alist.append(name)
        alist.append(author.text)
        for i in [-3,-2,-1]:
            alist.append(basic[i].text.replace("新功能介紹","").replace(" ","").replace("\n","").replace("出版社：","").replace("出版日期：","").replace("語言：",""))
        alist.append(price.text.replace("定價：",""))        
        return alist

    re2 = requests.get(url2)
    re2.encoding = 'utf-8'
    soup2 = BeautifulSoup(re2.text, 'html.parser')

    profile = soup2.find('div', {'class': 'PI_info'})
    name=profile.h1.a.text.replace("\n","")
    alist.append(name)
    
    basic=soup.find('td', {'class': 'summary'}).find_all('span')
    
    for c in [0,1,2,3]:
        if c==3:
            alist.append(basic[c].a.text.replace("著","").replace("出版日期:",""))   
        else:
            alist.append(basic[c].text.replace("著","").replace("出版日期:","").replace("出版股份有限公司","").replace("股份有限公司","").replace("有限公司",""))   
    price=profile.find('span',{'class':'price'})
    alist.append(price.text+"元")
    return alist

#輸入ISBN串列，得到書名簡介的Table
#呼叫checkprofile時同時填入字典中
def return_table_list(ISBN_list):
    table_list=[]
    for c in ISBN_list:
        d=checkprofile(c)
        table_list.append(d)
        profile_dic[c]=d
        time.sleep(1)
    return table_list

#分別print出三個排行榜的Table
#創造一個字典是ISBN對應書本資料
profile_dic={}
field_names=["書名","作者","出版社","出版日期","語言","定價"]    
print("博客來排行榜")
df1=pd.DataFrame(return_table_list(books_ISBN), columns=field_names) 
df1.index = range(1,11)
df1

博客來排行榜


,書名,作者,出版社,出版日期,語言,定價
1,完勝冰箱常備菜: 一口氣做起來放! 快速、簡單、變化多端,nozomi,悅知文化,2018/12/19,中文書,380元
2,完勝冰箱常備菜: 一口氣做起來放! 快速、簡單、變化多端,nozomi,悅知文化,2018/12/19,中文書,380元
3,完勝冰箱常備菜: 一口氣做起來放! 快速、簡單、變化多端,nozomi,悅知文化,2018/12/19,中文書,380元
4,完勝冰箱常備菜: 一口氣做起來放! 快速、簡單、變化多端,nozomi,悅知文化,2018/12/19,中文書,380元
5,完勝冰箱常備菜: 一口氣做起來放! 快速、簡單、變化多端,nozomi,悅知文化,2018/12/19,中文書,380元
6,完勝冰箱常備菜: 一口氣做起來放! 快速、簡單、變化多端,nozomi,悅知文化,2018/12/19,中文書,380元
7,完勝冰箱常備菜: 一口氣做起來放! 快速、簡單、變化多端,nozomi,悅知文化,2018/12/19,中文書,380元
8,完勝冰箱常備菜: 一口氣做起來放! 快速、簡單、變化多端,nozomi,悅知文化,2018/12/19,中文書,380元
9,完勝冰箱常備菜: 一口氣做起來放! 快速、簡單、變化多端,nozomi,悅知文化,2018/12/19,中文書,380元
10,完勝冰箱常備菜: 一口氣做起來放! 快速、簡單、變化多端,nozomi,悅知文化,2018/12/19,中文書,380元


In [0]:
print("誠品排行榜")
df2=pd.DataFrame(return_table_list(eslite_ISBN), columns=field_names) 
df2.index = range(1,11)
df2

誠品排行榜


,書名,作者,出版社,出版日期,語言,定價
1,"誰說一百分的妳, 才是最好的自己",Melody (殷悅),大塊文化,2018/12/03,中文書,350元
2,蔡康永的情商課: 為你自己活一次,蔡康永,如何出版社,2018/11/01,中文書,350元
3,解答之書: 專屬於你的人生答案,卡羅．波特,三采文化,2018/11/30,中文書,450元
4,八年執政回憶錄,馬英九/ 口述; 蕭旭岑,遠見天下文化,2018/12/20,中文書,630元
5,"99%的人輸在不會表達: 話說對了, 事就成了。公司裡該怎麼說話? 麻煩就沒了。",李勁,大是文化,2018/06/28,中文書,340元
6,成為這樣的我: 蜜雪兒．歐巴馬,蜜雪兒．歐巴馬,商業周刊,2018/11/15,中文書,560元
7,"連卡內基也佩服的7堂超溫暖說話課: 為何辯才無礙很吃虧? 因為要贏得感情, 你得輸點道理!",山崎拓巳,大樂文化,2018/03/14,中文書,280元
8,原則: 生活和工作,瑞．達利歐,商業周刊,2018/04/10,中文書,600元
9,只想回到你身邊,布魯斯．卡麥隆,圓神出版社,2018/06/01,中文書,350元
10,我是留德華: 海獅的德國奇幻旅程,神奇海獅李博研,聯經出版事業,2018/12/14,中文書,350元


In [0]:
print("金石堂排行榜")
df3=pd.DataFrame(return_table_list(kingstone_ISBN), columns=field_names) 
df3.index = range(1,11)
df3

金石堂排行榜


,書名,作者,出版社,出版日期,語言,定價
1,"一台筆電, 年收百萬",傅靖晏Terry Fu,創見文化,2017/08/09,中文書,350元
2,蔡康永的情商課: 為你自己活一次,蔡康永,如何出版社,2018/11/01,中文書,350元
3,"你好, 我是接體員",大師兄,寶瓶文化事業,2018/12/12,中文書,320元
4,NA,NA,NA,NA,NA,NA
5,今宵異譚 卷五 (完),四隻腳,三日月書版,2018/12/12,中文書,220元
6,成為這樣的我: 蜜雪兒．歐巴馬,蜜雪兒．歐巴馬,商業周刊,2018/11/15,中文書,560元
7,Sword Art Online刀劍神域Progressive 5,川原礫,台灣角川,2018/12/19,中文書,220元
8,NA,NA,NA,NA,NA,NA
9,解答之書: 專屬於你的人生答案,卡羅．波特,三采文化,2018/11/30,中文書,450元
10,"精進潛意識: 砍掉你的隱形負思維, 奪回命運自主權",哈利．卡本特,大寫出版,2018/07/05,中文書,280元


In [0]:
# 輸入三平台前十名的串列，得到每本書(ISBN)的排名得分
def weighted_rank(a):
    count=len(a)
    for x in a:
        if x in point_dic:
            point_dic[x]+=count*5
        else:
            point_dic[x]=count*5
        count-=1 

point_dic={}
weighted_rank(books_ISBN)
weighted_rank(eslite_ISBN)
weighted_rank(kingstone_ISBN)

#根據加權分數，產出前五名ISBN的串列
def dic(key):
    return point_dic[key]

def get_top_ten_ISBN(a):
    top_ten=[]
    a=sorted(a, reverse=True, key=dic)
    for x in a:
        if len(top_ten)==10:
            break
        elif x in top_ten or x=='xxxxxxxxxxxxx':
            continue
        else:
            top_ten.append(x)
    return top_ten

Total_ISBN_list=eslite_ISBN+kingstone_ISBN+books_ISBN
top_ten_ISBN=get_top_ten_ISBN(Total_ISBN_list)
print(top_ten_ISBN)

['9789578787704', '9789861365145', '9789862139318', '9789576580833', '9789867778444', '9789862717752', '9789864061419', '9789864796038', '9789579164337', '9789863616108']


In [0]:
# 輸入ISBN，回傳[博客來、價錢、庫存、連結]的串列

def books(key):
    alist=["博客來"]
  
    url = 'https://search.books.com.tw/search/query/key/'+key+'/cat/BKA'
    re = requests.get(url)
    re.encoding = 'utf-8'
    soup = BeautifulSoup(re.text, 'html.parser')

    first_result = soup.find('div', {'class': 'input_buy'})
    try: 
        url2= 'https://www.books.com.tw/products/'+first_result.input['value'] 
    except AttributeError:
        return["博客來","N/A","N/A","N/A"]
        
    url2= 'https://www.books.com.tw/products/'+first_result.input['value'] 

    re2 = requests.get(url2)
    re2.encoding = 'utf-8'
    soup2 = BeautifulSoup(re2.text, 'html.parser')

    profile = soup2.find('div', {'class': 'container_24 main_wrap clearfix'}).find('div',{'class':'grid_10'})
    price_profile=profile.find('div', {'class': 'cnt_prod002 clearfix'})
    price=price_profile.find_all('strong',{'class':'price01'})
    alist.append('TWD'+price[len(price)-1].text+"元")

    url3="https://www.books.com.tw/product_show/getProdCartInfoAjax/"+first_result.input['value']+"/M201105_032_view"
    re3 =requests.get(url3)
    re3.encoding = 'utf-8'
    soup3 = BeautifulSoup(re3.text, 'html.parser')

    stock= soup3.find('li',{'class':'no'})
    alist.append(stock.text)    
    alist.append(url2)
    
    return alist

In [0]:
# 輸入ISBN，回傳[金石堂、價錢、庫存、連結]的串列

def kingstone(keyword):
    kw2 = quote(keyword)
    kw2 = kw2.replace('%','%25')
    
    headers = {
    'user-agent':'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.94 Safari/537.36'
}
    url = 'https://www.kingstone.com.tw/search/result.asp?c_name='+kw2+'&se_type=4&sr1=館別&sr2=中文書'

    re = requests.get(url,headers=headers)
    soup = BeautifulSoup(re.text, 'html.parser')
    first_result = soup.find('div',{'class':'box row_list'})
    
    check = first_result.ul.a['href'].split('/')
    global new_link
    if check[1] == 'book':
        new_link = first_result.ul.a['href'].replace('book','basics')
        new_link = new_link.replace('_index','')
    
    elif check[1] == '3hrs':
        new_link = first_result.ul.a['href'].replace('3hrs_index','book_page')
    
    new_links = new_link.split('?')[0]
    book_id = first_result.ul.img['src'].split('/')[7]
    url2 = 'https://www.kingstone.com.tw'+new_links+'?kmcode='+book_id+'&'+new_link.split('?')[1]

    re2 = requests.get(url2,headers=headers)
    soup2 = BeautifulSoup(re2.text, 'html.parser')

    ks_list=['金石堂']
    price = soup2.find('b',{'class':'dis_p'})
    ks_list.append('TWD'+price.text+'元')

    if check[1] == 'book':
        inventory = soup2.find('div',{'class':'module01'})
        inventory = inventory.text.split('\xa0')
        inventory = inventory[0].split('立')
        ks_list.append(inventory[0].strip())
    elif check[1] == '3hrs':
        inventory = soup2.find('div',{'class':'module05'})
        ks_list.append(inventory.text.replace("\r","").replace("\n","").split('\t')[18])

    ks_list.append(url2)
    
    return ks_list


In [0]:
# 輸入ISBN，回傳[誠品、價錢、庫存、連結]的串列

def eslite(keyword):
    url = 'http://www.eslite.com/Search_BW.aspx?query='+keyword
    re = requests.get(url)
    re.encoding = 'utf-8'
    soup = BeautifulSoup(re.text, 'html.parser')
    first_result = soup.find('td', {'class': 'name'})
    url2 = 'http://www.eslite.com/'+first_result.h3.a['href'] 
    
    re2 = requests.get(url2)
    re2.encoding = 'utf-8'
    soup2 = BeautifulSoup(re2.text, 'html.parser')
    
    elite_list = ['誠品']
    price = soup2.find_all('span', {'class': 'price_sale'})
    elite_list.append('TWD'+price[1].text+'元')
    inventory = soup2.find('span',{'id':'ctl00_ContentPlaceHolder1_lblSaleStatusName'})
    elite_list.append(inventory.text)
    elite_list.append(url2)
    return(elite_list)

In [0]:
#搭配books/eslite/kingstone查價的function，抓出價格這個欄位

def takeout_price(x):
    return x[1]

#把takeout_price直接寫進比價function，可以直接以價格對books/eslite/kingstone三個查價結果排序

def compare_price(a):
    priceset = [books(a),eslite(a),kingstone(a)]
    priceorder = sorted(priceset, key=takeout_price,reverse=True)
    return priceorder

#需再將return的結果轉成table呈現
#此結果有兩種做法，一種是在加權排名後讓使用者input他要查哪一本
#或是直接帶入加權排行前五名，print出五個table


In [0]:
# 輸出加權後綜合排行榜
# 使用之前profile_dic字典與top_ten_ISBN串列

final_table_list=[]
for i in top_ten_ISBN:
    a=[profile_dic[i][0],profile_dic[i][1],profile_dic[i][2]]+compare_price(i)[0]
    final_table_list.append(a)
    time.sleep(10)

field_names_2=['排行榜書名',"作者","出版社","最優惠書店","最優惠價格","庫存量","訂購網址"]       
print("綜合排行榜")
df4=pd.DataFrame(final_table_list, columns=field_names_2) 
df4.index = range(1,11)
df4



綜合排行榜


,排行榜書名,作者,出版社,最優惠書店,最優惠價格,庫存量,訂購網址
1,完勝冰箱常備菜: 一口氣做起來放! 快速、簡單、變化多端,nozomi,悅知文化,博客來,TWD300元,庫存>10,https://www.books.com.tw/products/0010808229
2,蔡康永的情商課: 為你自己活一次,蔡康永,如何出版社,博客來,TWD277元,庫存>10,https://www.books.com.tw/products/0010802223
3,"誰說一百分的妳, 才是最好的自己",Melody (殷悅),大塊文化,博客來,TWD277元,庫存>10,https://www.books.com.tw/products/0010805561
4,解答之書: 專屬於你的人生答案,卡羅．波特,三采文化,博客來,TWD356元,購買後立刻進貨,https://www.books.com.tw/products/0010805313
5,成為這樣的我: 蜜雪兒．歐巴馬,蜜雪兒．歐巴馬,商業周刊,博客來,TWD442元,庫存>10,https://www.books.com.tw/products/0010803898
6,"一台筆電, 年收百萬",傅靖晏Terry Fu,創見文化,博客來,TWD315元,庫存>10,https://www.books.com.tw/products/0010761300
7,"你好, 我是接體員",大師兄,寶瓶文化事業,誠品,TWD253元,※ 有庫存可銷售,http://www.eslite.com/product.aspx?pgid=100114...
8,八年執政回憶錄,馬英九/ 口述; 蕭旭岑,遠見天下文化,誠品,TWD498元,※ 貨到後將立即為您出貨,http://www.eslite.com/product.aspx?pgid=100112...
9,"99%的人輸在不會表達: 話說對了, 事就成了。公司裡該怎麼說話? 麻煩就沒了。",李勁,大是文化,誠品,TWD269元,※ 貨到後將立即為您出貨,http://www.eslite.com/product.aspx?pgid=100119...
10,今宵異譚 卷五 (完),四隻腳,三日月書版,誠品,TWD187元,※ 有庫存可銷售,http://www.eslite.com/product.aspx?pgid=100130...
